# Real Data Analyses

In this section, we will explore applying spectral parameterization to a real dataset.

The data is from anesthesia, including a waking data segment, a low-dose propofol segment, and a high-dose propofol segment. See the data folder for more information on the data. 

The main goal here is to explore the real data, and use spectral parameterization to come to some conclusions about what is happening in the data, and what is changing between conditions. 

There are some prompts in this notebook to guide you towards some analyses you can apply. That said, these are meant as something of a starting point - you are encouraged to go beyond the prompts and to continue to explore this data.

In [ ]:
import os
from pathlib import Path

from scipy.io import loadmat

# Import fooof functions
from fooof import FOOOF
from fooof.plts import plot_spectrum
from fooof.analysis import get_band_peak_fm

# Import neurodsp functions
from neurodsp.filt import filter_signal
from neurodsp.spectral import compute_spectrum
from neurodsp.plts import plot_time_series, plot_power_spectra

## Load the Data

First, we will load some data. This section is filled in for you. 

In [ ]:
# Define the data path
data_path = Path('../data')

In [ ]:
# Get available data files
data_files_lst = [file for file in os.listdir(data_path) if '.mat' in file]
data_files_lst

### Data Information

EEG data from a healthy subject during propofol‑induced unconsciousness:
-  `eeganes07laplac250_detrend_all_t180.mat` Awake, before propofol induction
-  `eeganes07laplac250_detrend_all_t3070.mat` Unconscious, at a low dose of propofol
-  `eeganes07laplac250_detrend_all_t4710.mat` Unconscious, at a high dose of propofol

In [ ]:
# Get list of data file names, and store in a labelled dictionary
data_files_lst.sort()
data_files = {
    'awake' : data_files_lst[0],
    'low_dose' : data_files_lst[1],
    'high_dose' : data_files_lst[2],
}

### Load a Data File

To start, we will load a specific file to explore. 

If you want to switch which file you explore, change the 'condition' value below. 

In [ ]:
# Set which data file to load (change here to run another file)
condition = 'awake'   # 'awake', 'low_dose', 'high_dose'

In [ ]:
# Load a matfile with the data
matfile = loadmat(data_path / data_files[condition], squeeze_me=True)

In [ ]:
# Check the data keys that are the matfile
matfile.keys()

In [ ]:
# Extract data fields of interest
data = matfile['y'].T        # Data matrix - transpose to have channels as rows
fs = matfile['Fs']           # Sampling rate
times = matfile['ta']        # Times vector

In [ ]:
# For simplicity, reset times vector to go from 0 to 30
times = times - times[0]

### Extract channel of interest

To get started, we will choose a particular channel to examine. 

Channels 40 & 41 are frontal channels that are good channels to explore first, as they display features of interest and clear changes across the different brain states we have available. You should start with these channels - but feel free to explore other channels after. 

In [ ]:
# Define channel index of interest
chind = 41  # 40, 41

In [ ]:
# Extract channel data of interest
channel_data = data[chind, :]

## Data Explorations & Analyses

In this section, we will now start looking at and exploring / analyzing the data. 

From here, you need to start filling in code!

Note that to complete the prompts that are defined, you should not have to import any additional functions - however, you are more than welcome to import additional tools in order to extend the analyses. 

### Look at the time series

First, let's visualize the time series. 

In [ ]:
# Plot the time series
...

### Compute power spectrum

Next, let's compute a power spectra.

In [ ]:
# Compute power spectrum
...

In [ ]:
# Visualize power spectrum
...

### Parameterize the Spectrum

Next, let's fit a spectral model to the power spectrum we just computed.

In [ ]:
# Initialize spectral model
...

In [ ]:
# Fit spectral model to the data
...

#### Explore parameters

Now that we have a spectral model, explore the parameters that it fit. 

In particular, make sure to 

In [ ]:
# Extract the dominant low frequency peak
...

### Filter to the peak range

Next, let's explore the rhythm in the data. 

Filter the data around the frequency range of dominant peak (for example, +/- bandwidth around peak center frequency). 

In [ ]:
# Filter the signal to narrowband range
...

In [ ]:
# Visualize the narrowband filtered signal
...

In [ ]:
# Plot the filtered trace on top of the original data
...

In the above, we used sepctral parameterization to identify a peak of interest, and then examine this in the data. 

What does it look like when we do so:
- Is peak frequency as identified by spectral parameterization clearly present in the time series?
- If so, is the rhythm continuous / ongoing? Is it sinusoidal?
- If you answer no to either question above, what does this mean about how we should interpret / further analyze the data?

## Compare Data Across Brain States

Next, we will explore and compare the data across the different brain states.

Things you should try:
- Load and plot the time series for each state, to visually compare them
- Compute the power spectra across states, to visually compare them
- Fit spectral models across each state
- Compare the fit parameters, examing which parameters change across the different brain states

In [ ]:
# Check the available data files
data_files 

In [ ]:
...

## Conclusions

What have we learned about this dataset from using spectral parameterization?

Are there more aspects of the data we may want to investigate, that spectral parameterization does not do?